## Rioxarray with H5NetCDF and earthaccess


In [ ]:
import argparse

import earthaccess
import xarray as xr
from common import earthaccess_args
from rasterio.warp import calculate_default_transform

In [ ]:
def warp_resample(dataset):
    args = earthaccess_args[dataset]
    input_uri = f'{args["folder"]}/{args["filename"]}'
    src = f's3://{args["bucket"]}/{input_uri}'
    dstSRS = "EPSG:3857"
    srcSRS = "EPSG:4326"
    width = height = 256
    te = [
        -20037508.342789244,
        -20037508.342789244,
        20037508.342789244,
        20037508.342789244,
    ]

    fs = earthaccess.get_s3fs_session(daac=args["daac"])
    fsspec_caching = {
        "cache_type": "none",
    }
    with fs.open(src, **fsspec_caching) as f:
        da = xr.open_dataset(f, engine="h5netcdf", mask_and_scale=True)[
            args["variable"]
        ]
        if dataset == "gpm_imerg":
            da = da.rename({"lon": "x", "lat": "y"}).transpose("time", "y", "x")
        da = da.rio.write_crs(srcSRS)
        da = da.rio.clip_box(
            *te,
            crs=dstSRS,
        )
        dst_transform, w, h = calculate_default_transform(
            srcSRS,
            dstSRS,
            da.rio.width,
            da.rio.height,
            *da.rio.bounds(),
            dst_width=width,
            dst_height=height,
        )
        return da.rio.reproject(dstSRS, shape=(h, w), transform=dst_transform)

In [ ]:
if __name__ == "__main__":
    if "get_ipython" in dir():
        da = warp_resample("gpm_imerg")
    else:
        parser = argparse.ArgumentParser(description="Set environment for the script.")
        parser.add_argument(
            "--dataset",
            default="mursst",
            help="Dataset to resample.",
            choices=["gpm_imerg", "mursst"],
        )
        user_args = parser.parse_args()
        da = warp_resample(user_args.dataset)
